# Exercise 1

Use all feature selection methods to find the best features

## Dataset Information

## Features

Number of Instances: 20640

Number of Attributes: 8 numeric, predictive attributes and the target

Attribute Information:

MedInc - median income in block group

HouseAge - median house age in block group

AveRooms - average number of rooms per household

AveBedrms - average number of bedrooms per household

Population - block group population

AveOccup - average number of household members

Latitude - block group latitude

Longitude - block group longitude

## Target
The target variable is the median house value for California districts, expressed in hundreds of thousands of dollars ($100,000).

In [110]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import fetch_california_housing
from sklearn.feature_selection import VarianceThreshold
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [111]:
#loading the dataset

housing = fetch_california_housing(as_frame=True)
df = pd.concat([housing.data, housing.target], axis=1)

In [112]:
df_house = fetch_california_housing()
df_house_features = pd.DataFrame(df_house.data, columns=df_house.feature_names)
df_house_target = pd.DataFrame(df_house.target, columns=['target'])

In [113]:
df.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422


1. Use any filter method to select the best features

In [125]:
selector = VarianceThreshold(threshold=3) # Variance threshold
sel = selector.fit(df)
sel_index = sel.get_support()
df_norm_vt = df.iloc[:, sel_index]

In [126]:
df_norm_vt

,MedInc,HouseAge,AveRooms,Population,AveOccup,Latitude,Longitude
0,8.3252,41.0,6.984127,322.0,2.555556,37.88,-122.23
1,8.3014,21.0,6.238137,2401.0,2.109842,37.86,-122.22
2,7.2574,52.0,8.288136,496.0,2.802260,37.85,-122.24
3,5.6431,52.0,5.817352,558.0,2.547945,37.85,-122.25
4,3.8462,52.0,6.281853,565.0,2.181467,37.85,-122.25
...,...,...,...,...,...,...,...
20635,1.5603,25.0,5.045455,845.0,2.560606,39.48,-121.09
20636,2.5568,18.0,6.114035,356.0,3.122807,39.49,-121.21
20637,1.7000,17.0,5.205543,1007.0,2.325635,39.43,-121.22
20638,1.8672,18.0,5.329513,741.0,2.123209,39.43,-121.32


2. Use any wrapper method to select the best features

In [116]:
from sklearn.ensemble import RandomForestRegressor # Changed to Regressor
from sklearn.feature_selection import RFE

threshold = 5 # the number of most relevant features
model_rf = RandomForestRegressor(n_estimators=500, random_state=0, max_depth = 3) # Changed to Regressor
selector = RFE(model_rf, n_features_to_select=5, step=1)

selector = selector.fit(df_house_features, df_house_target.values.ravel())
selector_ind = selector.get_support()
df_wine_rfe = df_house_features.iloc[:, selector_ind]
df_wine_rfe.columns

Index(['MedInc', 'HouseAge', 'AveRooms', 'AveOccup', 'Latitude'], dtype='object')

3. Use any embedded methood to select the best features

In [117]:
from sklearn.feature_selection import SelectFromModel

model_rf = RandomForestRegressor(n_estimators=500, random_state=0, max_depth = 3)
model_rf.fit(df_house_features, df_house_target.values.ravel())

sel_sfm = SelectFromModel(model_rf, prefit=True)
sel_sfm_index = sel_sfm.get_support()
df_wine_sfm = df_house_features.iloc[:, sel_sfm_index]
df_wine_sfm.columns

Index(['MedInc', 'AveOccup'], dtype='object')

testing


In [118]:
# Assuming df_wine_sfm contains the selected features and df_house_target contains the target variable
X = df_wine_sfm  # Features
y = df_house_target  # Target variable

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train a RandomForestRegressor model
model = RandomForestRegressor(n_estimators=100, random_state=42) # You can adjust hyperparameters
model.fit(X_train, y_train.values.ravel())

# Make predictions on the test set
y_pred = model.predict(X_test)

# Calculate the RMSE
drmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"Default RMSE: {drmse}")

Default RMSE: 0.7986500926536819


In [128]:
# Select relevant features
selected_features = ['MedInc', 'HouseAge', 'AveRooms', 'Population', 'AveOccup', 'Latitude', 'Longitude', 'MedHouseVal']
df_selected = df[selected_features]

# Separate features (X) and target (y)
X = df_selected.drop('MedHouseVal', axis=1)
y = df_selected['MedHouseVal']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train a RandomForestRegressor model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Calculate RMSE
frmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"RMSE (filter method): {frmse}")

RMSE (filter method): 0.5039778571218967


In [120]:
# Select relevant features
selected_features = ['MedInc', 'HouseAge', 'AveRooms', 'AveOccup', 'Latitude', 'MedHouseVal']
df_selected = df[selected_features]

# Separate features (X) and target (y)
X = df_selected.drop('MedHouseVal', axis=1)
y = df_selected['MedHouseVal']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train a RandomForestRegressor model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Calculate RMSE
wrmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"RMSE (wrapper method): {wrmse}")

RMSE (wrapper method): 0.6080133261490511


In [121]:
# Select relevant features
selected_features = ['MedInc', 'AveOccup', 'MedHouseVal']
df_selected = df[selected_features]

# Separate features (X) and target (y)
X = df_selected.drop('MedHouseVal', axis=1)
y = df_selected['MedHouseVal']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train a RandomForestRegressor model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Calculate RMSE
ermse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"RMSE (embedded method): {ermse}")

RMSE (embedded method): 0.7986500926536819


In [129]:
print(f"Default RMSE: {drmse}")
print(f"RMSE (filter method): {frmse}")
print(f"RMSE (wrapper method): {wrmse}")
print(f"RMSE (embedded method): {ermse}")

Default RMSE: 0.7986500926536819
RMSE (filter method): 0.5039778571218967
RMSE (wrapper method): 0.6080133261490511
RMSE (embedded method): 0.7986500926536819
